In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
import tensorflow


def data_preprocess(file_location):
    nvda_data = pd.read_csv(file_location)

    print(nvda_data.head())

    nvda_data['Date'] = pd.to_datetime(nvda_data['Date'])

    # moving average
    nvda_data['MA_week'] = nvda_data['Close'].rolling(window=7).mean()
    nvda_data['MA_month'] = nvda_data['Close'].rolling(window=30).mean()

    # volume average
    nvda_data['VL_week'] = nvda_data['Volume'].rolling(window=7).mean()
    nvda_data['VL_month'] = nvda_data['Volume'].rolling(window=30).mean()

    nvda_data.dropna(inplace=True)

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(nvda_data[['Open', 'High', 'Low', 'Close', 'Volume']])

    nvda_scaled = pd.DataFrame(scaled_data, columns=['Open', 'High', 'Low', 'Close', 'Volume'])
    print(nvda_scaled)

    nvda_scaled['Next_Day_Close'] = nvda_scaled['Close'].shift(-1)

    x = nvda_scaled.drop('Next_Day_Close', axis=1)
    y = nvda_scaled['Next_Day_Close']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    modeling(x_train, y_train, y_test)


def modeling(x_training, y_training, y_test):
    model = Sequential()
    model.add(LSTM(units=50, input_shape=(x_training.shape[1], 1)))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_training.values.reshape((x_training.shape[0], x_training.shape[1], 1)), y_training, epochs=50,
              batch_size=32)

    validation(y_training, y_test)


def validation(y_test, prediction):
    plt.figure(figsize=(10, 6))
    plt.plot(y_test.values, label='Actual')
    plt.plot(prediction, label='Predicted')
    plt.title('Actual vs. Predicted Values')
    plt.legend()
    plt.show()


if __name__ == '__main__':
    data_preprocess(r'.\data\NVDA.csv')